# Original testing

In [ ]:
from utils import models
from utils.models import FioBase
import subprocess

In [ ]:
f = FioBase()
fio_params = [("--bs", "8k"), 
              ("--name", "job1"), 
              ("--ioengine","libaio"), 
              ("--iodepth", "1"), 
              ("--size","20m"), 
              ("--filename","/tmp/test.fio"), 
              ("--direct", "1"),
              ("--rw", "randread"),
              ("--runtime", "5"),
              "--group_reporting",
              "--time_based"]
param_list: list = ["--output-format=json"]
for param in fio_params:
    print(type(param))
    if type(param) == str:
        param_list.append(param)
    else:
        param_list.append(param[0] + "=" + param[1])
param_list

In [ ]:
fio_process = subprocess.run(["fio"] + param_list,
                                capture_output=True)
fio_process.stdout

In [ ]:
import json
from utils.fio_runner import FioRunner
json_result = json.loads(fio_process.stdout)
fio = FioRunner()
# dir(fio)
fiobase = fio.stdout_to_FioBase(fio_process.stdout)
str(fiobase)

# FioOptimizer Class
## from optimal-io-depth-search.py

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Notes

# fio.find_optimal_iodepth()
# a = fio.best_run.prepare_args(params=fio.config)
# fio_process = subprocess.run(['fio'] + [f"{k}={v}" if v else f"{k}" for k, v in fio.config.items()], capture_output=True)
# print(fio_process)

# param_list = [f"{k}={v}" if v else f"{k}" for k, v in fio.config.items()]
# print(param_list)
# fio_process = subprocess.run(['fio'] + param_list, capture_output=True)
# fio_process.stdout
# fio_run: FioBase = FioBase()
# fio_run.parse_stdout(raw_stdout=fio_process.stdout)
# print(fio.prepare_and_run_fio(io_depth=6))
# print(fio.prepare_and_run_fio(io_depth=8))
# print(fio.prepare_and_run_fio(io_depth=10))



#  Start at 1,40
# where do I go in the middle?
# slices = 5 
# range(1,40 abs(40-1)/slices)  39/5 = 7.x 1+7, 1+(7*2), 1+(7*3), 1+(7*4)

# Future TODO - version 1.5+
# determine protocol, 
# bundle fio with the repo
# system tunables (max tcp slot table entry, others)

# TODO (for MVP)
#   128*nconnect number
#   128*nr_smb_sessions
#   either total_iops as a ratio of latency OR look at delta of total_iops as compared to the delta of latency
#       if the latency_delta is larger than iops_delta then store smallest value of latency_delta  
#   option for max_io_rate
#   option for max_io_rate_before_knee_of_curve

In [ ]:
from utils import models
from utils.models import FioOptimizer, FioBase

fio_params = {"--bs": "8k", 
              "--name": "job1", 
              "--ioengine":"libaio", 
              "--iodepth": "1", 
              "--size": "20m", 
              "--filename": "./test.fio", 
              "--direct": "1",
              "--rw": "randread",
              "--runtime": "10s",
              "--time_based": None,
              "--group_reporting": None,
              "--output-format": "json",
              "--norandommap": None,
              "--dedupe_percentage": 0,
              "--buffer_compress_percentage": 50,
              "--buffer_compress_chunk": 4096,
              "--buffer_pattern": "\"aaaa\""}
param_list: list = [f"{k}={v}" for k, v in fio_params.items()]
# print(fio_params)
# print(param_list)

fio = FioOptimizer(config=fio_params, min=1, max= 40) 

# # print(f"Fio Config: {fio.config}")

is_optimial: bool = False

starting_io_depths: list = [fio.min, fio.max]    # gotta start some where

for io_depth in starting_io_depths:
    fio.config['--iodepth'] = io_depth
    # print(fio.config)
    fio_run = fio.prepare_and_run_fio(io_depths=[io_depth])
    fio.runs[io_depth] = fio_run


In [ ]:
fio.find_optimal_iodepth()
print(fio.best_run)

In [ ]:
print(fio.best_run)

##Notes

    # test limits (min, max)
    # test midpoint between limits
    # average results of min+midpoint and max+midpoint
    # 
    # divide total from min and max into n buckets, then go test all of them,
    # sort them 
    # range(min, max, (abs(max-min))/5)
    # slices = 5 
    # 10, 20, 2 # ((20-10)/slices = 2)
    # k = [10, 12, 14, 16, 18]
    # when I do the loop, test if k[n] is already tested
    # find the best results from min, max, k[]
    

    # while not is_optimial:
    #     sorted_runs_by_iops: list[FioBase] = sorted(self.runs, key=self.runs.get('total_iops'), reverse=True)
    #     # are we going up or down?
    #     if sorted_runs_by_iops[0].io_depth - sorted_runs_by_iops[1].io_depth <= 1:
    #         is_optimial = True
    #         self.optimal_queue_depth = sorted_runs_by_iops[0]
    #         return 
    #     else: 
    #         next_io_depth = ((sorted_runs_by_iops[0].io_depth + sorted_runs_by_iops[1].io_depth) // 2)
    #         if next_io_depth in self.runs:
    #             next_io_depth = 
    #         else:
    #             self.prepare_and_run_fio(io_depth=next_io_depth)
    